In [ ]:
from prettytable import PrettyTable
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [ ]:
from datetime import date, timedelta
import json
import requests


class DefaultFilter:
    def filter(self, data):
        return data


class ReFilter(DefaultFilter):
    def __init__(self, pattern, key):
        self.key = key
        self.pattern = pattern
        self.re_pattern = re.compile(pattern)

    def filter(self, data):
        return [row for row in data if self.re_pattern.search(row[key])]


class DefaultReader:
    def __init__(self):
        self.format = "xml"

    def decode(self):
        return request.content


class JsonReader:
    def __init__(self):
        self.format = "json"

    def decode(self, request):
        return json.loads(request.content)


class Matomo:
    def __init__(self, reader=DefaultReader, period=None, filter=None, **init_params):
        self.url = "https://stats.data.gouv.fr/index.php"
        self.parameters = {
            "module": "API",
            "method": "Actions.getPageUrls",
            "idSite": 236,
            "period": "day",
            "date": "today",
            "token_auth": "e13428c4dc3dcb05b0b5cb0cbbf31714",
            "expanded": 1,
            "flat": 1,
        }
        self.set_period(*period)
        self.parameters.update(init_params)
        self.reader = reader()
        self.parameters.update({"format": self.reader.format})
        if filter:
            self.filter = filter
        else:
            self.filter = lambda x: x

    def set_period(self, *args):
        if args is None:
            return
        start, end = min(args), max(args)
        self.parameters.update({
            "period": "range",
            "date": f"{start.strftime('%Y-%m-%d')},{end.strftime('%Y-%m-%d')}"
        })

    def send(self):
        request = requests.get(self.url, params=self.parameters)
        return self.reader.decode(request)

In [ ]:
mato = Matomo(reader=JsonReader, period=(date(2022, 6, 20), date(2022, 6, 1)))
data = mato.send()
data

In [ ]:
x = PrettyTable(field_names=["page", "visits", "visiteurs"], align="l")
x.add_rows((row["label"][:75], row["nb_hits"], row["nb_visits"]) for row in data)
print(x)

Plan de taggage

page => indicateur
index => visites de l'app

In [ ]:
import re
from collections import defaultdict

re_map = re.compile(r"(consommation|couverture|synthesis|usage|artificialisation)")

total = defaultdict(lambda: 0)
for row in data:
    match = re_map.search(row["label"])
    if match:
        total[match.group(0)] += row["nb_hits"]
        

x = PrettyTable(field_names=["Rapport", "Hits"], align="l")
x.add_rows((k, v) for k, v in total.items())
print(x)

In [ ]:
[row["label"] for row in data]

In [ ]:
re_map.search("/project/2396/map").group(0)

In [ ]:
today = date.today()
today

In [ ]:
first = date(year=today.year - 1, month=today.month + 1, day=1)
first

In [ ]:
from django.db.models import Subquery, OuterRef, F
from project.models import Request, Emprise
from public_data.models import Region
from utils.colors import get_color_gradient

project_list = Request.objects.all().values_list("project", flat=True)
emprise_list = 
qs_regions = Region.objects.all().annotate(total=Subquery(emprise_list[:1])).order_by("total")
colors = get_color_gradient(scale=qs.regions.count())
for region in qs_regions:
    region.map_color = colors.pop(0)
for region in qs_regions:
    print(region.name, region.map_color)

In [ ]:
project_list = Request.objects.all().values_list("project", flat=True)
sub_qs = Emprise.objects.filter(mpoly__contained=OuterRef("mpoly")).annotate(total=Count("project_id", distinct=True)).values("total")[:1]
qs = (
    Region.objects
    .annotate(total=Subquery(sub_qs))
    .order_by("total")
)

for region in qs:
    print(region.name, region.total)

In [ ]:
Emprise.objects.filter(
    # project__in=project_list,
    mpoly__intersects=region.mpoly
).annotate(total=Count("project_id", distinct=True)).values("total")[:1]

In [ ]:
Request.objects.all().values_list("project", flat=True)